In [1]:
import json
import csv

In [2]:
input_file = 'data/flu_data.txt'
output_file = 'data/google_flu.json'

In [3]:
country_names = []
with open("data/world-110m-country-names.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    for row in rd:
#         print(row[1])
        country_names.append(row[1])
country_names = set(country_names)

In [4]:
country_names 

{'Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antarctica',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia, Plurinational State of',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Congo, the Democratic Republic of the',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Falkland Islands (Malvinas)',
 'Fiji',
 'Finland',
 'France',
 'French Southern Territories',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Greenland',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Ho

In [5]:
data_names = []
with open(input_file) as fd:
    rd = csv.reader(fd, delimiter=",")
    for row in rd:
        if row[0] == 'Date':
            data_names = row
            data_names.remove('Date')
            data_names = set(data_names)
            break

In [6]:
data_names

{'Argentina',
 'Australia',
 'Austria',
 'Belgium',
 'Bolivia',
 'Brazil',
 'Bulgaria',
 'Canada',
 'Chile',
 'France',
 'Germany',
 'Hungary',
 'Japan',
 'Mexico',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Paraguay',
 'Peru',
 'Poland',
 'Romania',
 'Russia',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'Ukraine',
 'United States',
 'Uruguay'}

In [7]:
def string_convert(input_string):
    if input_string == 'Bolivia':
        return 'Bolivia, Plurinational State of'
    elif input_string == 'Russia':
        return 'Russian Federation'
    else:
        return input_string

In [8]:
for data_name in data_names:
    if string_convert(data_name) not in country_names:
        print(data_name)

In [9]:
country_ids = {}
with open("data/world-110m-country-names.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t")
    for row in rd:
        if row[1] == 'name':
            continue
        country_id = int(row[0])
        country_name = row[1]
        country_ids[country_name] = country_id

In [10]:
dictionary = {}
country_mapping = {}
with open(input_file) as fd:
    rd = csv.reader(fd, delimiter=",")
    prev_year = 2002
    for row in rd:
        if row[0] == 'Date':
            dictionary['countries'] = {}
            for i in range(1,len(row)):
                if string_convert(row[i]) not in dictionary['countries']:
                    country_mapping[i] = string_convert(row[i])
                    dictionary['countries'][string_convert(row[i])] = {}
                    dictionary['countries'][string_convert(row[i])]['name'] = string_convert(row[i])
                    dictionary['countries'][string_convert(row[i])]['years'] = {}
                    if string_convert(row[i]) in country_ids:
                        dictionary['countries'][string_convert(row[i])]['id'] = country_ids[string_convert(row[i])]
                    else:
                        dictionary['countries'][string_convert(row[i])]['id'] = -9999
        else:
            if int(row[0].split('-')[0]) != prev_year:
                prev_year = int(row[0].split('-')[0])
                
            date = row[0].split('-')[1] + '-' + row[0].split('-')[2]
            
            for i in range(1,len(row)):
                
                if prev_year not in dictionary['countries'][country_mapping[i]]['years']:
                    dictionary['countries'][country_mapping[i]]['years'][prev_year] = {}
                    
                dictionary['countries'][country_mapping[i]]['years'][prev_year][date] = {}
                if row[i] == '':
                    dictionary['countries'][country_mapping[i]]['years'][prev_year][date]['weekly influenza activity estimates for the country'] = 0.0
                else:
                    dictionary['countries'][country_mapping[i]]['years'][prev_year][date]['weekly influenza activity estimates for the country'] = float(row[i])
                    

In [11]:
dictionary

{'countries': {'Argentina': {'id': 32,
   'name': 'Argentina',
   'years': {2003: {'01-05': {'weekly influenza activity estimates for the country': 0.0},
     '01-12': {'weekly influenza activity estimates for the country': 0.0},
     '01-19': {'weekly influenza activity estimates for the country': 0.0},
     '01-26': {'weekly influenza activity estimates for the country': 0.0},
     '02-02': {'weekly influenza activity estimates for the country': 136.0},
     '02-09': {'weekly influenza activity estimates for the country': 145.0},
     '02-16': {'weekly influenza activity estimates for the country': 141.0},
     '02-23': {'weekly influenza activity estimates for the country': 135.0},
     '03-02': {'weekly influenza activity estimates for the country': 134.0},
     '03-09': {'weekly influenza activity estimates for the country': 136.0},
     '03-16': {'weekly influenza activity estimates for the country': 134.0},
     '03-23': {'weekly influenza activity estimates for the country': 15

In [12]:
with open(output_file, 'w') as fp:
    json.dump(dictionary, fp)